In [1]:
import pandas as pd

# Read the file with all sheets
f = pd.ExcelFile('ctcnDataInput.xlsx')

# Sheets contains the list of countries a.k.a. sheet names
sheets = f.sheet_names

In [2]:
dfs = []

# For all sheets, cleanse the data
for c in range(len(sheets)):
    
    # Load the specific sheet
    df = pd.read_excel('ctcnDataInput.xlsx', sheet_name=sheets[c])
    
    # Drop rows that are all blank
    df = df.dropna(axis='rows',how='all')
    
    # Reset the index for simplicity
    df = df.reset_index(drop=True)
    
    # Clean up first column. Create aseries with that column
    parentSeries = df.iloc[:,0]
    # Convert everything to upper case for simplicity
    parentSeries = parentSeries.str.upper()
    # Fill all nan values with 0. Nan values != undefined.
    parentSeries = parentSeries.fillna(0)
    x = parentSeries.tolist()
    
    if 'PARENT CATEGORY' not in x:
        print('Missing Parent Category cell. Check sheet for ', sheets[c])
    else:
        # if there are values in category column only then do processing:
        if len(x)>0:

            # 'Unmerge' the values from list
            catVal = 0
            # Set catVal as the value from parent category if its not 0 or parent category. Else set that element to catVal.
            for el in range(len(x)):
                # Set catVal and use that to set all data values in subsequent sections
                if x[el] != 0:
                    if x[el] != 'PARENT CATEGORY':
                        catVal = x[el]
                else:
                    x[el] = catVal

            # Create a parent category column in df.
            df['Parent Cat'] = x

            # Drop all rows above first Mitigation cell in first column
            df = df.drop(list(range(x.index('PARENT CATEGORY')+1)))

            # Drop the first column because it has bad data
            cName = df.columns.values[0]
            df = df.drop(labels=cName,axis=1)

            # Rearrange DF columns and rename all columns to remove undefined values.
            testC = df.columns.tolist()
            # The right sheets have 10 columns.
            if len(testC)!=10:
                print('{} has {} columns, should have 10. Check sheet.'.format(sheets[c],len(testC)))
            else:
                testC = testC[-1:]+testC[:-1]
                df = df[testC]
                colNames = ['Parent Category','Category', 'INDC Sector', 'INDC technology',	'Unconditional Target', 'Technology Needs',\
                            'More information on INDC technology','TNA / TAP technology prioritised','More information of on TNA / TAP Technology',\
                            'Actions / project ideas']
                df.columns = colNames

                # Fill nan values for downstream ease
                # We have dropped all empty rows by now.
                df = df.fillna(0)

                # Category series can also have 0s, so lets 'unmerge' this column
                tempL = df['Category'].tolist()
                for s in range(len(tempL)):
                    if tempL[s] != 0:
                        # first element
                        priorVal = tempL[s]
                    else:
                        # encountered a nan value, set equal to priorVal
                        tempL[s]=priorVal

                # Update Category column with values
                df['Category'] = tempL

                # Add country as a column
                df['Country'] = [sheets[c]]*df.shape[0]

                dfs.append(df)
        # Else the file we got has no parent category and something is off with it.
        else:
            print('Parent Category column for {} has issues'.format(sheets[c]))
            print(x)

compiledDF = pd.concat(dfs,ignore_index=True)
# Drop rows with column headers
indToDrop = compiledDF.loc[compiledDF['Unconditional Target']=='Unconditional Target'].index.tolist()
compiledDF = compiledDF.drop(index=indToDrop)
# Remove anywhere the indc technology was 0. Any undefined cells should be labeled undefined so this is just a list of nans that we converted to 0.
newDF = compiledDF.loc[compiledDF['INDC technology']!=0]
newDF.to_csv('processed0823.csv')

In [3]:
sheets[c]

'Zimbabwe'

*****************************

In [4]:
import pandas as pd

In [5]:
# Load taxonomy information, this is what we compare against.
taxonomy = pd.read_csv('taxonomy.csv')
ctcnList = taxonomy['INDC Tech'].tolist()
ctcnList.append('Undefined')

In [6]:
# Load entered values.
loadDF = pd.read_csv('processed0823.csv', index_col=0)
enteredValues = loadDF['INDC technology'].tolist()

In [7]:
# Strip white spaces
valsWoWhitespace = []
for v in enteredValues:
    valsWoWhitespace.append(v.strip())

In [8]:
# Make everything capitalized
# Create an array for entered values in all capitals
capsRemVals = [x.upper() for x in valsWoWhitespace]
# Convert CTCN list to all capitals too
capsCtcnList = [x.upper() for x in ctcnList]

In [9]:
# Deal with innovative values
innovList = []
for v in capsCtcnList:
    if '*' in v:
        innovList.append(v.replace(' *',''))

In [10]:
innovList

['CO2 CAPTURE TECHNOLOGIES',
 'INTELLIGENT TRANSPORT SYSTEMS',
 'HIGH-SPEED RAIL',
 'CAR SHARING',
 'ADVANCED BIOFUELS',
 'HYBRID ELECTRIC VEHICLES',
 'HYDROGEN INFRASTRUCTURE',
 'SMART GRID',
 'FUEL CELL TECHNOLOGIES',
 'ENERGY HARVESTING',
 'CARBON SINK AND LOW-CARBON BUILDING MATERIALS',
 'DESIGN FOR DECONSTRUCTION',
 'PASSIVE HOUSE DESIGN',
 'ZERO-ENERGY BUILDING',
 'SUPERCONDUCTING MAGNETIC ENERGY STORAGE',
 'SOLAR TOWERS (UPDRAFT)',
 'ENERGY TOWER (DOWNDRAFT)',
 'SOLAR LED LIGHTS',
 'FUEL CELLS',
 'GRID INTEGRATION FOR RENEWABLES',
 'HYBRID RENEWABLE ENERGY SYSTEMS',
 'TIDAL ENERGY',
 'BIOFUELS FROM ALGAE',
 'WET COMPOSTING',
 'PEER-TO-PEER LENDING PLATFORMS',
 'PRODUCT-AS-SERVICE',
 'LIFE CYCLE THINKING',
 'DESIGN FOR DISASSEMBLY',
 'BIOCHAR',
 'CCS FROM CEMENT PRODUCTION',
 'INDUSTRIAL SYMBIOSIS',
 'ECO-DESIGN',
 'TRIGENERATION',
 'DISASTER RISK ASSESSMENT TOOLS',
 'CLIMATE-RESILIENT TREES',
 'INDIGENOUS FOREST MANAGEMENT PRACTISES',
 'SOILLESS AGRICULTURE',
 'BIOCHAR TECHNOLOG

In [11]:
# Compare entered values with CTCN list and innovative values
invL = []
for v in capsRemVals:
    tempVal = -1
    
    # takes care of capitalization errors and leading/trailing spaces
    if v in capsCtcnList:
        tempVal = 1
    else:
        # takes care of missing ' *' in technologies
        if v in innovList:
            tempVal = 1
        else:
            tempVal = 0
    invL.append(tempVal)

In [12]:
# update the values in main DF
loadDF['INDC technology'] = capsRemVals

In [13]:
loadDF['Valid?'] = invL

In [14]:
loadDF.loc[loadDF['Valid?']==0].shape

(289, 12)

In [15]:
loadDF.loc[loadDF['Valid?']==1].shape

(2631, 12)

In [16]:
ct = 0
for v in capsRemVals:
    if v in innovList:
        loadDF.iloc[ct,3] = v+' *'
        print(loadDF.iloc[ct,3])
    ct += 1

HYBRID ELECTRIC VEHICLES *
HIGH-SPEED RAIL *
INTELLIGENT TRANSPORT SYSTEMS *
CO2 CAPTURE TECHNOLOGIES *
GRID INTEGRATION FOR RENEWABLES *
HYBRID ELECTRIC VEHICLES *
INDIGENOUS FOREST MANAGEMENT PRACTISES *
GRID INTEGRATION FOR RENEWABLES *
GRID INTEGRATION FOR RENEWABLES *
CLIMATE-RESILIENT TREES *
DISASTER RISK ASSESSMENT TOOLS *
CLIMATE INSURANCE *
DISASTER RISK ASSESSMENT TOOLS *
CROP INSURANCE *
CLIMATE INSURANCE *
SEAWATER DESALINATION *
HYBRID ELECTRIC VEHICLES *
ADVANCED BIOFUELS *
CLIMATE-RESILIENT TREES *
HYBRID RENEWABLE ENERGY SYSTEMS *
GRID INTEGRATION FOR RENEWABLES *
HYBRID ELECTRIC VEHICLES *
CLIMATE INSURANCE *
HYBRID ELECTRIC VEHICLES *
SEAWATER DESALINATION *
TIDAL ENERGY *
GRID INTEGRATION FOR RENEWABLES *
HIGH-SPEED RAIL *
LIFE CYCLE THINKING *
DISASTER RISK ASSESSMENT TOOLS *
SEAWATER DESALINATION *
SEAWATER DESALINATION *
FUEL CELLS *
GRID INTEGRATION FOR RENEWABLES *
ADVANCED BIOFUELS *
SEAWATER DESALINATION *
SEAWATER DESALINATION *
DISASTER RISK ASSESSMENT TOOL

In [17]:
inputData = loadDF.loc[loadDF['Valid?']==1]

In [18]:
pcList = inputData['Parent Category'].tolist()

In [19]:
cleanPcList = []
for v in pcList:
    cleanPcList.append(v.replace('\n',''))

In [20]:
inputData['Parent Category'] = cleanPcList

C:\Users\bhavrathod\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
inputData.head()

,Parent Category,Category,INDC Sector,INDC technology,Unconditional Target,Technology Needs,More information on INDC technology,TNA / TAP technology prioritised,More information of on TNA / TAP Technology,Actions / project ideas,Country,Valid?
0,MITIGATION,Energy Efficiency,Appliances and equipment,APPLIANCE STANDARDS,0,Carbon fiance and project development skills. ...,0,0,0,0,Afghanistan,1
1,MITIGATION,Energy Efficiency,Buildings,BUILDING STANDARDS AND CODES,0,0,0,2 - Small Hydropower,potential for hydropower development estimated...,Reduce dependence on imported electricity. Imp...,Afghanistan,1
2,MITIGATION,Energy Efficiency,Undefined,UNDEFINED,0,0,0,0,0,0,Afghanistan,1
3,MITIGATION,Renewable Energy,Undefined,UNDEFINED,0,Human and institutional capacity for adoption ...,0,0,0,0,Afghanistan,1
4,MITIGATION,Renewable Energy,Solar,SOLAR PV,0,0,0,Energy Sector - prioritization - 1 - Solar Ene...,"Large-scale, - The main constraint to the depl...",0,Afghanistan,1


In [22]:
inputData.to_csv('validData.csv')

In [23]:
loadDF.loc[loadDF['Valid?']==0].to_csv('invalid.csv')

************